In [1]:
import pandas  as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


train=pd.read_csv("C:\\Users\\Prerit\\Desktop\\projects\\Artificial Intelligence- Deep Learning\\NLP\\NLP disaster tweets\\data\\train.csv")
test=pd.read_csv("C:\\Users\\Prerit\\Desktop\\projects\\Artificial Intelligence- Deep Learning\\NLP\\NLP disaster tweets\\data\\test.csv")

In [2]:
rows,columns=map(int,train.shape)
print("Number of rows =" ,rows,"\nNumber of columns =",columns,"\n")

print(train.isnull().any(),"\n\n")

null_columns=train.columns[train.isnull().any()] #Stores all columns which have null types in them
#print(null_columns)

print(train[null_columns].isnull().sum(),"\n\n")

Number of rows = 7613 
Number of columns = 5 

id          False
keyword      True
location     True
text        False
target      False
dtype: bool 


keyword       61
location    2533
dtype: int64 




In [3]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')     #To convert into a list of words (in this step we only create our tokenizer and pass regex argument)
    tokens = tokeniser.tokenize(text)       #We then use our tokenizer to tokenize the text

    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    print(keywords)
    return keywords

In [ ]:
count_vectorizer = TfidfVectorizer(analyzer=preprocess_text)
# count_vectorizer = feature_extraction.text.CountVectorizer(analyzer=preprocess_text) This is the count vectorizer
train_vectors = count_vectorizer.fit_transform(train["text"])

test_vectors = count_vectorizer.transform(test["text"])

In [33]:
clf = linear_model.RidgeClassifier()

[0.61371841 0.53839802 0.6006192  0.57639524 0.70316114]


In [43]:
sgd_clf = linear_model.SGDClassifier()
sgf_clf_scores = model_selection.cross_val_score(sgd_clf, train_vectors, train['target'], cv=5,scoring="f1")
print(sgf_clf_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (sgf_clf_scores.mean(), sgf_clf_scores.std() * 2))

[0.60900901 0.54111842 0.59782609 0.57923497 0.70588235]
Accuracy: 0.61 (+/- 0.11)


In [23]:
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=3, scoring="f1")
scores

array([0.61555433, 0.58732612, 0.67639658])

In [34]:
sgd_clf.fit(train_vectors, train["target"])

SGDClassifier()

In [ ]:
grid = {'fit_intercept': [True,False],
        'early_stopping': [True, False],
        'loss' : ['hinge', 'log_loss', 'squared_hinge'],
        'penalty' : ['l2', 'l1', 'none']}
search = model_selection.GridSearchCV(estimator=clf, param_grid=grid, cv=3)
search.fit(train_vectors, train["target"])
search.best_params_

In [55]:
grid_sgd_clf_scores = model_selection.cross_val_score(search.best_estimator_, train_vectors, train['target'], cv=5)
print(grid_sgd_clf_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (grid_sgd_clf_scores.mean(), grid_sgd_clf_scores.std() * 2))

[0.71306632 0.67629678 0.69271175 0.70170828 0.78055191]
Accuracy: 0.71 (+/- 0.07)


In [13]:
sample_submission =pd.read_csv("C:\\Users\\Prerit\\Desktop\\projects\\Artificial Intelligence- Deep Learning\\NLP\\NLP disaster tweets\\data\\sample_submission.csv")

In [52]:
sample_submission["target"] = search.best_estimator_.predict(test_vectors)

In [53]:
sample_submission.to_csv("submissions\\submission_Ridgeclassifier_TdIdf_girdSearchCV_cv=3.csv", index=False)

In [30]:
print(train_vectors[0].todense())

(1, 19620)
